# Create a Simple Neural Search Service


This tutorial shows you how to build and deploy your own neural search service to look through descriptions of companies from startups-list.com and pick the most similar ones to your query. The website contains the company names, descriptions, locations, and a picture for each entry.


A neural search service uses artificial neural networks to improve the accuracy and relevance of search results. Besides offering simple keyword results, this system can retrieve results by meaning. It can understand and interpret complex search queries and provide more contextually relevant output, effectively enhancing the user’s search experience.



## Workflow
To create a neural search service, you will need to transform your raw data and then create a search function to manipulate it. First, you will 1) download and prepare a sample dataset using a modified version of the BERT ML model. Then, you will 2) load the data into Qdrant, 3) create a neural search API and 4) serve it using FastAPI.


Note: The code for this tutorial can be found here: | Step 1: Data Preparation Process | Step 2: Full Code for Neural Search. |




## Prerequisites
To complete this tutorial, you will need:

Docker - The easiest way to use Qdrant is to run a pre-built Docker image.
Raw parsed data from startups-list.com.
Python version >=3.8


## Prepare sample dataset
To conduct a neural search on startup descriptions, you must first encode the description data into vectors. To process text, you can use a pre-trained models like BERT or sentence transformers. The sentence-transformers library lets you conveniently download and use many pre-trained models, such as DistilBERT, MPNet, etc.

First you need to download the dataset.
```bash
wget https://storage.googleapis.com/generall-shared-data/startups_demo.json
```

In [4]:
%pip install sentence-transformers numpy pandas tqdm torch torchvision torchaudio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch

print(torch.cuda.is_available())
print(torch.backends.mps.is_available())
device = torch.device("mps")  # Metal Performance Shaders(MPS)를 사용

False
True


In [14]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

model = SentenceTransformer(
    "all-MiniLM-L6-v2", device="cpu"
)  # or device="cpu" if you don't have a GPU

df = pd.read_json("./startups_demo.json", lines=True)

df

,name,images,alt,description,link,city
0,SaferCodes,https://safer.codes/img/brand/logo-icon.png,SaferCodes Logo QR codes generator system form...,QR codes systems for COVID-19.\nSimple tools f...,https://safer.codes,Chicago
1,Human Practice,https://d1qb2nb5cznatu.cloudfront.net/startups...,Human Practice - health care information tech...,Point-of-care word of mouth\nPreferral is a mo...,http://humanpractice.com,Chicago
2,StyleSeek,https://d1qb2nb5cznatu.cloudfront.net/startups...,StyleSeek - e-commerce fashion mass customiza...,Personalized e-commerce for lifestyle products...,http://styleseek.com,Chicago
3,Scout,https://d1qb2nb5cznatu.cloudfront.net/startups...,Scout - security consumer electronics interne...,Hassle-free Home Security\nScout is a self-ins...,http://www.scoutalarm.com,Chicago
4,Invitation codes,https://invitation.codes/img/inv-brand-fb3.png,Invitation App - Share referral codes community,The referral community\nInvitation App is a so...,https://invitation.codes,Chicago
...,...,...,...,...,...,...
40469,Drunken Moose,https://d1qb2nb5cznatu.cloudfront.net/startups...,Drunken Moose - digital media advertising des...,Branding and Marketing Consultancy Agency\nHel...,http://www.drunkenmoose.com.au,Sydney
40470,AA Adonis Rubbish Removals,https://d1qb2nb5cznatu.cloudfront.net/startups...,AA Adonis Rubbish Removals - cleaning,Rubbish Removals Sydney\nAA Adonis Rubbish Rem...,http://www.aaadonisrubbishremovals.com.au/,Sydney
40471,QualityTrade,https://d1qb2nb5cznatu.cloudfront.net/startups...,QualityTrade - B2B,Merit based wholesale trade platform. \nQualit...,https://www.qualitytrade.com/,Sydney
40472,The Myer Family Company,https://d1qb2nb5cznatu.cloudfront.net/startups...,The Myer Family Company -,MFCo is a family office specialising in design...,http://www.mfco.com.au/,Sydney


In [15]:
vectors = model.encode(
    [row.alt + ". " + row.description for row in df.itertuples()],
    show_progress_bar=True,
)


Batches: 100%|██████████| 1265/1265 [01:51<00:00, 11.31it/s]


In [16]:
vectors.shape

(40474, 384)

In [17]:
np.save("startup_vectors.npy", vectors, allow_pickle=False)

## Run Qdrant in Docker

Next, you need to manage all of your data using a vector engine. Qdrant lets you store, update or delete created vectors. Most importantly, it lets you search for the nearest vectors via a convenient API.

Note: Before you begin, create a project directory and a virtual python environment in it.

Download the Qdrant image from DockerHub.

```bash 
docker pull qdrant/qdrant
```

Start Qdrant inside of Docker.

```bash 
docker run -p 6333:6333 \
    -v $(pwd)/qdrant_storage:/qdrant/storage \
    qdrant/qdrant
```
Test the service by going to http://localhost:6333/. You should see the Qdrant version info in your browser.

All data uploaded to Qdrant is saved inside the ./qdrant_storage directory and will be persisted even if you recreate the container.





## Upload data to Qdrant
Install the official Python client to best interact with Qdrant.


In [18]:
%pip install qdrant-client

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [19]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient("http://localhost:6333")

In [20]:
client.recreate_collection(
    collection_name="startups",
    vectors_config=VectorParams(size=384, distance=Distance.COSINE),
)

/var/folders/r4/w6gk0qbd6bd_sf7xj6nwdnxc0000gn/T/ipykernel_22875/2024731105.py:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

In [21]:
fd = open("./startups_demo.json")

# payload is now an iterator over startup data
payload = map(json.loads, fd)

# Load all vectors into memory, numpy array works as iterable for itself.
# Other option would be to use Mmap, if you don't want to load all data into RAM
vectors = np.load("./startup_vectors.npy")

In [22]:
vectors

array([[-0.10995458,  0.05200215, -0.05883856, ...,  0.05058498,
         0.00743647, -0.00689396],
       [-0.01339604,  0.05021113, -0.04075696, ..., -0.05376014,
         0.10636391,  0.05142042],
       [-0.11035899,  0.0826499 , -0.01236237, ..., -0.10485063,
         0.00271895,  0.01831402],
       ...,
       [-0.02728407, -0.04229014, -0.06790321, ..., -0.0885737 ,
         0.10970899,  0.04764627],
       [-0.00196704, -0.03750686, -0.06574629, ..., -0.01514671,
         0.01988766, -0.03325766],
       [-0.0443297 ,  0.08066224,  0.02030026, ..., -0.11605135,
        -0.05278013,  0.00159997]], dtype=float32)

In [23]:
client.upload_collection(
    collection_name="startups",
    vectors=vectors,
    payload=payload,
    ids=None,  # Vector ids will be assigned automatically
    batch_size=256,  # How many vectors will be uploaded in a single request?
)

## Build the search API
Now that all the preparations are complete, let’s start building a neural search class.

In order to process incoming requests, neural search will need 2 things: 1) a model to convert the query into a vector and 2) the Qdrant client to perform search queries.

Create a file named neural_searcher.py and specify the following.


In [27]:
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer


class NeuralSearcher:
    def __init__(self, collection_name):
        self.collection_name = collection_name
        # Initialize encoder model
        self.model = SentenceTransformer("all-MiniLM-L6-v2", device="cpu")
        # initialize Qdrant client
        self.qdrant_client = QdrantClient("http://localhost:6333")

    def search(self, text: str):
        # Convert text query into vector
        vector = self.model.encode(text).tolist()

        # Use `vector` for search for closest vectors in the collection
        search_result = self.qdrant_client.search(
            collection_name=self.collection_name,
            query_vector=vector,
            query_filter=None,  # If you don't want any filters for now
            limit=5,  # 5 the most closest results is enough
        )
        # `search_result` contains found vector ids with similarity scores along with the stored payload
        # In this function you are interested in payload only
        payloads = [hit.payload for hit in search_result]
        return payloads

## Add search filters.
With Qdrant it is also feasible to add some conditions to the search. For example, if you wanted to search for startups in a certain city, the search query could look like this:



In [31]:
from qdrant_client.models import Filter

city_of_interest = "Berlin"

# Define a filter for cities
city_filter = Filter(**{
    "must": [{
        "key": "city", # Store city information in a field of the same name 
        "match": { # This condition checks if payload field has the requested value
            "value": city_of_interest
        }
    }]
})

# search_result = client.search(
#     collection_name="startups",
#     query_vector=vector,
#     query_filter=city_filter,
#     limit=5
# )

## Deploy the search with FastAPI
To build the service you will use the FastAPI framework.

Install FastAPI.
To install it, use the command



In [32]:
%pip install fastapi uvicorn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached fastapi-0.111.0-py3-none-any.whl.metadata (25 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached starlette-0.37.2-py3-none-any.whl.metadata (5.9 kB)
  Using cached fastapi_cli-0.0.4-py3-none-any.whl.metadata (7.0 kB)
  Using cached python_multipart-0.0.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached ujson-5.10.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (9.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
  Using cached email_validator-2.2.0-py3-none-any.whl.metadata (25 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached dnspython-2.6.1-py3-none-any.whl.metadata (5.8 kB)
  Using cached typer-0.12.3-py3-none-any.whl.metadata (15 kB)
  Using cached httptools-0.6.1-cp312-cp312-macosx_10_9_universal2.whl.metadata (3.6 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached uvloop-0.19.0-cp312-cp312-macosx_10_9_universal2.whl.metadata (4.9 kB

In [34]:
from fastapi import FastAPI

app = FastAPI()

# Create a neural searcher instance
neural_searcher = NeuralSearcher(collection_name="startups")


@app.get("/api/search")
def search_startup(q: str):
    return {"result": neural_searcher.search(text=q)}


if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="0.0.0.0", port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop